## Chi-squared test или хи-квадрат

Разберем хи-квадрат на примере классического АБ теста

In [46]:
import numpy as np
import pandas as pd
from scipy import stats as st

#### Гипотеза и данные

Пусть мы тестируем конверсию в клик на двух разных страничках. На версию А и Б пришли по 1000 пользователей, но разное количество пользователей кликнуло.

H0: Конверсия A не отличается от конверсии B

H1: Конверсия A отличается от конверсии B


In [3]:
#Данные
ab_data = np.array([[900, 100], [930, 70]])
ab_data

array([[900, 100],
       [930,  70]])

In [4]:
# Превратим эти данные в таблицу сопряженности. 

#Сначала добавим тоталы по строкам
row_total = np.sum(ab_data, axis=1)
row_total = row_total.reshape((len(row_total),1))
contingency_table = np.append(ab_data, row_total, axis=1)

# Теперь по столбцам
column_total = np.sum(contingency_table, axis=0)
contingency_table = np.append(contingency_table, [column_total], axis=0)

contingency_table

array([[ 900,  100, 1000],
       [ 930,   70, 1000],
       [1830,  170, 2000]])

Чтобы проверить нулевую гипотезу нам нужны ожидаемые значения, где мы предполагаем, что между А и Б нет разницы

In [27]:
expected_values = np.empty(shape=ab_data.shape)
for i, row in enumerate(ab_data):
    for j, column in enumerate(row):
        expected_value = contingency_table[i,-1]/contingency_table[-1,-1] * contingency_table[-1,j]
        expected_values[i, j] = expected_value
expected_values

array([[915.,  85.],
       [915.,  85.]])

#### Проверка условий для проведения теста

1. Условие случайно выборки

Мы проходим это условие, так как рандомно показывали пользователям страницу А или страницу Б.
2. Ожидаемое значение > 5

Здесь все хорошо. Ожидаемые значения в таблице expected_values больше пят
3. Условие независимости (sampling with replacement или размер выборки меньше 10% от популяции)

Тут тоже проходим. 

Теперь посчитаем нашу статистику

In [31]:
chi_squared_statistic = np.sum((ab_data - expected_values)**2 / expected_values)
chi_squared_statistic

5.785920925747348

In [34]:
# степень свободы
df = (ab_data.shape[0] - 1) * (ab_data.shape[1] - 1)
df

1

In [37]:
# Для расчета p_value используем кумулятивное распределение хи-квадрат.
# Ищем вероятность получения значения статистики или значения выше этой статистики при заданой df.
p_value = 1 - st.chi2.cdf(chi_squared_statistic, df)
p_value

0.01615503207374036

In [39]:
if p_value < 0.05:
    print('Отвергаем нулевую гипотезу, p-value:{}'.format(p_value))
else:
    print('Не можем отвергнуть нулевую гипотезу, p-value:{}'.format(p_value))

Отвергаем нулевую гипотезу, p-value:0.01615503207374036


#### Проверка с помощью scipy

In [45]:
chi2, p, dof, ex = st.chi2_contingency(ab_data, correction=False)
p

0.01615503207374036